### The Capabilities of PyQASM 
Besides unrolling and semantic validation, PyQASM also provides a variety of functionalities to help users analyse and transform their quantum programs. We will highlight them by taking a look at a simple example. 

For detailed documentation, please refer to the [official PyQASM documentation](https://sdk.qbraid.com/projects/pyqasm/en/stable/api/pyqasm.html).

In [ ]:
%pip install pyqasm --quiet

In [1]:
import pyqasm

pyqasm.__version__

'0.1.0'

In [2]:
from pyqasm import loads, dumps

In [3]:
qasm_str = """
OPENQASM 3.0;
include "stdgates.inc";

// define qubits and random ops 
qubit[10] q;
bit[10] c;

h q[0];
h q[1];

cx q[0], q[1];

rx(0.5) q[3:6];
ccx q[0], q[1], q[2];

barrier q[:5];

c[:5] = measure q[:5]; 
"""

In [4]:
module = loads(qasm_str)

- Let's start by validating our quantum program.

In [5]:
module.validate()

- Next, we will get the qubit and classical register information. Post this, we will unroll the program and print it.

In [6]:
module.num_qubits, module.num_clbits

(10, 10)

In [7]:
module.unroll()

dumps(module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[10] q;',
 'bit[10] c;',
 'h q[0];',
 'h q[1];',
 'cx q[0], q[1];',
 'rx(0.5) q[3];',
 'rx(0.5) q[4];',
 'rx(0.5) q[5];',
 'ccx q[0], q[1], q[2];',
 'barrier q[0];',
 'barrier q[1];',
 'barrier q[2];',
 'barrier q[3];',
 'barrier q[4];',
 'c[0] = measure q[0];',
 'c[1] = measure q[1];',
 'c[2] = measure q[2];',
 'c[3] = measure q[3];',
 'c[4] = measure q[4];']

In [8]:
module.depth()

5

- Let us now remove the idle qubits from our qasm program and get back the optimized qasm code.
- Note that `q[6]` to `q[9]` are idle qubits in the above qasm code.

In [9]:
module.remove_idle_qubits()

In [10]:
dumps(module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[6] q;',
 'bit[10] c;',
 'h q[0];',
 'h q[1];',
 'cx q[0], q[1];',
 'rx(0.5) q[3];',
 'rx(0.5) q[4];',
 'rx(0.5) q[5];',
 'ccx q[0], q[1], q[2];',
 'barrier q[0];',
 'barrier q[1];',
 'barrier q[2];',
 'barrier q[3];',
 'barrier q[4];',
 'c[0] = measure q[0];',
 'c[1] = measure q[1];',
 'c[2] = measure q[2];',
 'c[3] = measure q[3];',
 'c[4] = measure q[4];']

In [11]:
module.num_qubits, module.num_clbits

(6, 10)

- We can clearly see that the size of the quantum program has decreased to 6 qubits instead of 10
- Let's check for measurements and barriers now

In [12]:
module.has_measurements()

True

In [13]:
module.has_barriers()

True

- Barriers and measurements can easily be removed from the qasm code through the `remove_barriers()` and `remove_measurements()` functions respectively.
- We can also use the `in_place` parameter to specify whether we want to modify the original qasm code or not.

In [14]:
module.remove_barriers()

new_module = module.remove_measurements(in_place=False)

In [15]:
dumps(module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[6] q;',
 'bit[10] c;',
 'h q[0];',
 'h q[1];',
 'cx q[0], q[1];',
 'rx(0.5) q[3];',
 'rx(0.5) q[4];',
 'rx(0.5) q[5];',
 'ccx q[0], q[1], q[2];',
 'c[0] = measure q[0];',
 'c[1] = measure q[1];',
 'c[2] = measure q[2];',
 'c[3] = measure q[3];',
 'c[4] = measure q[4];']

In [16]:
dumps(new_module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[6] q;',
 'bit[10] c;',
 'h q[0];',
 'h q[1];',
 'cx q[0], q[1];',
 'rx(0.5) q[3];',
 'rx(0.5) q[4];',
 'rx(0.5) q[5];',
 'ccx q[0], q[1], q[2];']

- We can observe that the barriers have been removed from both the module objects but measurements are only removed in the `new_module`


- We also have the ability to **reverse the qubit order**

In [17]:
module.reverse_qubit_order()

In [18]:
dumps(module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[6] q;',
 'bit[10] c;',
 'h q[5];',
 'h q[4];',
 'cx q[5], q[4];',
 'rx(0.5) q[2];',
 'rx(0.5) q[1];',
 'rx(0.5) q[0];',
 'ccx q[5], q[4], q[3];',
 'c[0] = measure q[5];',
 'c[1] = measure q[4];',
 'c[2] = measure q[3];',
 'c[3] = measure q[2];',
 'c[4] = measure q[1];']

- Qubits have been reversed in place in the qasm code 


- We can finally check for the number of qubits, classical bits and depth of the updated qasm code 

In [19]:
module.num_qubits, module.num_clbits

(6, 10)

In [20]:
module.depth()

4